In [8]:
import random
import time
import uuid
import numpy as np
import json

# List of vehicle ids
vehicleIds = []

# List of events:
event_type_list = [
    "speeding", # event_definitions.speeding,
    "massive_speeding", # event_definitions.massive_speeding,
    "cruise_control_activated", # event_definitions.cruise_control_activated,
    "tcs_activated", # event_definitions.tcs_activated,
    "esc_activated", # event_definitions.esc_activated,
    "performance_mode_activated", # event_definitions.performance_mode_activated,
    "autobahn", # event_definitions.autobahn,
    "traffic_jam", # event_definitions.traffic_jam,
    "no_seatbelt", # event_definitions.no_seatbelt,
    "harsh_braking", # event_definitions.harsh_braking,
    "harsh_acceleration", # event_definitions.harsh_acceleration,
    "harsh_cornering", # event_definitions.harsh_cornering,
]

# Converting list to numpy array
event_type_array = np.array(event_type_list)

# Generates a list of 10 vehicle ids 
for i in range(10):
    vehicleId = str(uuid.uuid4())
    vehicleIds.append(vehicleId)

# Set the initial start date and time
import datetime
start_time = "2024-02-21T13:09:10.288+01:00"

# Add radom number of days to the start_ts
start_ts = datetime.datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S.%f%z")

print(start_ts)
print(type(start_ts))


def generate_sample_data(vehicleId, start_ts):
    data = []

    # Set the trip Id:
    tripId = str(uuid.uuid4())

    # Generate a random number of days between 1 and 30
    random_days = random.randint(1, 30)

    # Set the initial ts for the first iteration and create a new ramdon ts in the future for the next iterations
    initial_ts = start_ts + datetime.timedelta(days=random_days)
    current_ts = initial_ts

    # Generate 10 events for the current vehicle
    for i in range(10):
        
        if i > 1:
            # Generate a random number of minutes between 1 and 10
            random_minutes = random.randint(1, 10)
            current_ts = current_ts + datetime.timedelta(minutes=random_minutes)
   
        riskEventData = {
            "tripId": tripId,
            "vehicleId": vehicleId,
            "name": np.random.choice(event_type_array),
            "eventId": str(uuid.uuid4()),
            "riskLevel": random.randint(1, 5),
            "timestamp": str(current_ts),
            "eventData": json.dumps({"Vehicle_Speed_Speed": round(random.uniform(130, 150), 2)})
        }
        data.append(riskEventData)
    return data


# events to be generated
events = []

# Generates a list of events for each vehicle
for vehicleId in vehicleIds:
    for i in range(10):
    
        event = generate_sample_data(vehicleId, start_ts)
        events.append(event)

events_flat_list = [item for sublist in events for item in sublist]

print(type(events_flat_list))

#for e in events_flat_list:
#    print(e)

StatementMeta(, a3b92c9d-4a10-44e9-bd4c-65da763b4be1, 12, Finished, Available)

2024-02-21 13:09:10.288000+01:00
<class 'datetime.datetime'>
<class 'list'>


In [20]:
Vehicle_Acceleration_Longitudinal = [round(random.uniform(-3, -1), 4) for _ in range(5)]
Vehicle_Speed_Speed_innerorts = [round(random.uniform(30, 50), 0) for _ in range(5)]
Vehicle_Speed_Speed_100 = [round(random.uniform(80, 100), 0) for _ in range(5)]
Vehicle_Speed_Speed_autobahn = [round(random.uniform(100, 129), 0) for _ in range(5)]
Chassis_Brake_Pressure = [round(random.uniform(1, 11), 1) for _ in range(5)]
Chassis_SteeringWheel_Angle = [round(random.uniform(97, 106), 1) for _ in range(5)]
Vehicle_Acceleration_Lateral = [round(random.uniform(-0.3, -0.5), 2) for _ in range(5)]
ADAS_ABS_Error = [round(random.uniform(0.0, 0.2), 2) for _ in range(5)]
ADAS_ABS_IsEngaged  = [round(random.uniform(0.0, 0.2), 2) for _ in range(5)]
ADAS_TCS_IsEngaged  = [round(random.uniform(0.0, 0.2), 2) for _ in range(5)]

for e in Vehicle_Acceleration_Lateral:
    print(e)

StatementMeta(, a3b92c9d-4a10-44e9-bd4c-65da763b4be1, 24, Finished, Available)

-0.44
-0.47
-0.46
-0.38
-0.42


In [14]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("tripId", StringType(), True),
    StructField("vehicleId", StringType(), True),
    StructField("name", StringType(), True),
    StructField("eventId", StringType(), True),
    StructField("riskLevel", IntegerType(), True),
    StructField("timestamp", StringType(), True),
    StructField("eventData", StringType(), True)
])

rdd = spark.sparkContext.parallelize(events_flat_list)

df = spark.read.json(rdd, schema=schema)

display(df)

StatementMeta(, a3b92c9d-4a10-44e9-bd4c-65da763b4be1, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, a7f0a78e-9677-46bf-aecd-b91306763e57)

In [15]:
# Write DataFrame to CSV
# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
timestamp = now.strftime("%Y%m%d_%H%M")

# Add the timestamp to the filename
filename = f"test_events_{timestamp}.csv"
df.repartition(1).write.format('csv').option('header',True).mode('overwrite').save('Files/dev/sample_data/risk_events_raw/' + filename)



StatementMeta(, a3b92c9d-4a10-44e9-bd4c-65da763b4be1, 19, Finished, Available)